In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pandas as pd, seaborn as sns
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm

In [3]:
WIDE_index = pd.read_csv('WIDE_index_association.csv')
WIDE_photom = fits.open('../refWIDE_onRecovered/ref_sample/photometry.fits')
photom_data = WIDE_photom[1].data

Read label things

In [118]:
label_sample = 'WIDE'
ref_sample_path = 'ref_sample_{}'.format(label_sample)
original_ref_sample_path = '../ref{}_onRecovered/ref_sample/'.format(label_sample)

MAMBO_catalog = fits.open('../../../_catalogues/MAMBO_NDR1_{}_magcut.fits'.format(label_sample))[1].data
MAMBO_df = Table(MAMBO_catalog)[['ID', 'z_app', 'M', 'SFR']].to_pandas()

label_index = fits.open('{}/posteriors/Index_File_posterior.fits'.format(original_ref_sample_path))
MAMBO_id = label_index[1].data['OBJECT_ID']
label_index = pd.DataFrame(np.load('{}/index.npy'.format(original_ref_sample_path)))
label_index['MAMBO_id'] = MAMBO_id
label_index['MAMBO_id'] = label_index['MAMBO_id'].astype('int')

label_index = label_index.merge(MAMBO_df.rename(columns = {'ID': 'MAMBO_id'}))

The concept is: substitute all the z, M and SFR posterior samples with the true values taken from the MAMBO simulated catalog.

In [137]:
pp_paths = sorted([original_ref_sample_path+p for p in os.listdir(original_ref_sample_path.format(label_sample)) if p.startswith('pp')])
pp_files = [np.load(p) for p in pp_paths]

In [139]:
for idx in range(len(pp_files)):
    print(idx)
    
    arr = label_index[label_index['pp_file'] == idx+1]['z_app'].values
    pp_files[idx]['REDSHIFT'] = np.tile(arr.reshape(1, len(arr)), (100, 1)).T

    arr = label_index[label_index['pp_file'] == idx+1]['M'].values
    pp_files[idx]['STELLARMASS'] = np.tile(arr.reshape(1, len(arr)), (100, 1)).T

    arr = np.log10(label_index[label_index['pp_file'] == idx+1]['SFR'].values)
    pp_files[idx]['SFR'] = np.tile(arr.reshape(1, len(arr)), (100, 1)).T

0
1
2
3
4
5


In [142]:
np.save('ref_sample/pp_data_1.npy', pp_files[0])
np.save('ref_sample/pp_data_2.npy', pp_files[1])
np.save('ref_sample/pp_data_3.npy', pp_files[2])
np.save('ref_sample/pp_data_4.npy', pp_files[3])
np.save('ref_sample/pp_data_5.npy', pp_files[4])
np.save('ref_sample/pp_data_6.npy', pp_files[5])

In [153]:
index = label_index[['id', 'weight', 'pdz_file', 'pdz_offset', 'pp_file', 'pp_offset', 'sed_file', 'sed_offset']].to_records(index=False)
np.save('ref_sample/index.npy', index)

---